In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import xarray as xr
import torch

## Input: 
'data/byrd_bedmachine.nc'
## Output 1: 
3D tensor (3, 400, 400) [s, x, y] 'data/surface_m_ellipsoid_true_s_x_y.pt'
## Output 2: 
flat csv pd.DataFrame columns [s, x, y] 'data/surface_m_ellipsoid_true_s_x_y_flat.csv'

In [2]:
path_to_file_server = 'data/byrd_bedmachine.nc'
byrd_bedmachine = xr.open_dataset(path_to_file_server)

## Use ellipsoid data and get true surface elevation (REMA)

In [3]:
byrd_bedmachine["surface_ellipsoid"] = byrd_bedmachine["surface"] + byrd_bedmachine["geoid"]
byrd_bedmachine["surface_ellipsoid_true"] = byrd_bedmachine["surface_ellipsoid"] + byrd_bedmachine["firn"]

## Tensor 

- 3 2D fields

In [4]:
# grid x and y data
xx, yy = np.meshgrid(byrd_bedmachine.x.values, byrd_bedmachine.y.values)

In [8]:
byrd_bedmachine["surface_ellipsoid_true"].values

data = np.concatenate((
    byrd_bedmachine["surface_ellipsoid_true"].values.reshape(1, 400, 400), 
    xx.reshape(1, 400, 400), 
    yy.reshape(1, 400, 400)))

print(data.shape)

(3, 400, 400)


In [9]:
torch.save(torch.tensor(data), "data/surface_m_ellipsoid_true_s_x_y.pt")

# csv pd.DataFrame

- flat 
- use for merging

In [17]:
# transpose needed
byrd_surface = pd.DataFrame(data.reshape(3, -1).T, columns = ('s', 'x', 'y'))
byrd_surface

,s,x,y
0,2136.356445,350000.0,-800500.0
1,2131.236816,350500.0,-800500.0
2,2129.025879,351000.0,-800500.0
3,2127.416504,351500.0,-800500.0
4,2126.574951,352000.0,-800500.0
...,...,...,...
159995,2067.705078,547500.0,-1000000.0
159996,2074.884766,548000.0,-1000000.0
159997,2085.696777,548500.0,-1000000.0
159998,2095.212646,549000.0,-1000000.0


In [18]:
byrd_surface.to_csv('data/surface_m_ellipsoid_true_s_x_y_flat.csv')